In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import re

In [ ]:
data = pd.read_csv("https://github.com/Hemachandra151/Datasets/raw/main/sexual%20harassmnt.csv")
data.head()

,text,label
0,I was walking alone in the park when a group o...,1
1,"While waiting at the bus stop, a man standing ...",1
2,I was walking down a deserted road at night wh...,1
3,"At the market place, a vendor kept staring at ...",1
4,A colleague at work keeps cracking jokes about...,1


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1004 entries, 0 to 1003
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1004 non-null   object
 1   label   1004 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [ ]:
data.describe()

,label
count,1004.000000
mean,0.516932
std,0.499962
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


#Train Test Split

In [ ]:
x,tx = train_test_split(data, random_state=2529)
x = x.values.tolist()

#Data Preprocessing

In [ ]:
label = ['No', 'Yes']

nx = []
yx = []


for v in x:
  if v[1]==0:
    st = v[0].lower()
    nx = nx + re.findall(r'\b\w+\b', st)
  else:
    st = v[0].lower()
    yx.extend(re.findall(r'\b\w+\b', st))


frequency_n = {}
frequency_y = {}

for element in nx:
    if element in frequency_n:
        frequency_n[element] += 1
    else:
        frequency_n[element] = 1

for element in yx:
    if element in frequency_y:
        frequency_y[element] += 1
    else:
        frequency_y[element] = 1

#Navie Bayes Classifier

In [ ]:
def isSexualHarassement(st):
  input = re.findall(r'\b\w+\b', st)

  res = [1,1]

  for w in input:
    if w in frequency_y:
      p = frequency_y[w]
      q=p
      if w in frequency_n:
        q = q + frequency_n[w]
      res[1] = res[1]*p/q
    if w in frequency_n:
      p = frequency_n[w]
      q=p
      if w in frequency_y:
        q = q + frequency_y[w]
      res[0] = res[0]*p/q

  res[0] = res[0] * len(nx)/(len(nx)+len(yx))
  res[1] = res[1] * len(yx)/(len(nx)+len(yx))

  return label[np.argmax(res)]

In [ ]:
val = isSexualHarassement("A spouse surprises their partner with flowers and a romantic dinner.")
print(val)

No


#Testing The Model

In [ ]:
tx = tx.values.tolist()

conf = np.zeros((2,2))

for v in tx:
  if isSexualHarassement(v[0]) == label[v[1]]:
    if label[v[1]]=='No':
      conf[0][0]+=1
    else:
      conf[1][1]+=1
  else:
    if label[v[1]]=='No':
      conf[1][0]+=1
    else:
      conf[0][1]+=1

print(f"Out of {conf[0][0]+conf[1][0]} labled as No {conf[0][0]} were correctly predicted")
print(f"Out of {conf[1][1]+conf[0][1]} labled as Yes {conf[1][1]} were correctly predicted")
print(conf[0][0]+conf[1][1])
print(conf[0][0]+conf[0][1]+conf[1][0]+conf[1][1])
print((conf[0][0]+conf[1][1])*100/(conf[0][0]+conf[0][1]+conf[1][0]+conf[1][1]))

Out of 115.0 labled as No 83.0 were correctly predicted
Out of 136.0 labled as Yes 121.0 were correctly predicted
204.0
251.0
81.27490039840637
